In [ ]:
!pip install tensorflow streamlit nltk

In [ ]:
import pandas as pd

data = pd.read_csv('Shakespeare_data.csv')
text = data['PlayerLine'].str.cat(sep=' ')

In [ ]:
import string
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
# Preprocessing
text = text.translate(str.maketrans('', '', string.punctuation)).lower().split()

In [ ]:
unique_words = set(text)
print(f'Total unique words: {len(unique_words)}')

Total unique words: 27381


In [ ]:
tokenizer = Tokenizer(num_words=15000)
tokenizer.fit_on_texts([text])  # Fit tokenizer on text
sequences = tokenizer.texts_to_sequences([text])[0]  # Convert text to sequences

In [ ]:
# Vocabulary size
total_words = len(tokenizer.word_index) + 1  # Adding 1 for padding (if needed)
print(f'Total words in vocabulary: {total_words}')

Total words in vocabulary: 27382


In [ ]:
# Get the word index and calculate total unique words
word_index = tokenizer.word_index
total_unique_words = min(len(word_index), 15000)  # Ensure it doesn't exceed 15000

print("Total unique words:", total_unique_words)

Total unique words: 15000


In [ ]:
import numpy as np
# Define the length of input sequences (sliding window size)
sequence_length = 5  

max_sequences = 10000

input_sequences = []
next_words = []

for i in range(sequence_length, min(len(sequences), max_sequences)):
    input_sequences.append(sequences[i-sequence_length:i])
    next_words.append(sequences[i])

# Convert lists to numpy arrays
X = np.array(input_sequences)
y = np.array(next_words)

print(f"Input sequences: {X}")
print(f"Next words: {y}")

Input sequences: [[ 307    3  132    3  807]
 [   3  132    3  807    1]
 [ 132    3  807    1  568]
 ...
 [ 171 1947 1048   62  171]
 [1947 1048   62  171   59]
 [1048   62  171   59   43]]
Next words: [  1 568  63 ...  59  43   8]


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.utils import to_categorical

In [ ]:
# One-hot encode the target labels (next words)
y = to_categorical(y, num_classes=total_unique_words)
y

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
print(len(y))

9995


In [ ]:
# Build the LSTM model
model = Sequential()
model.add(Embedding(input_dim=total_words, output_dim=10, input_length=sequence_length))
model.add(LSTM(100, return_sequences=False))
model.add(Dense(total_unique_words, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=50, batch_size=4)

Epoch 1/50
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 25s 9ms/step - accuracy: 0.0339 - loss: 7.4492
Epoch 2/50
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 24s 9ms/step - accuracy: 0.0330 - loss: 6.2042
Epoch 3/50
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 23s 9ms/step - accuracy: 0.0431 - loss: 6.0128
Epoch 4/50
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 23s 9ms/step - accuracy: 0.0575 - loss: 5.8296
Epoch 5/50
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 23s 9ms/step - accuracy: 0.0552 - loss: 5.6163
Epoch 6/50
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 23s 9ms/step - accuracy: 0.0646 - loss: 5.3284
Epoch 7/50
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 23s 9ms/step - accuracy: 0.0718 - loss: 5.0838
Epoch 8/50
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 23s 9ms/step - accuracy: 0.0820 - loss: 4.8169
Epoch 9/50
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 23s 9ms/step - accuracy: 0.1097 - loss: 4.4913
Epoch 10/50
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 23s 9ms/step - accuracy: 0.1539 - loss: 4.1406
Epoch 11/50
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 23s 9ms/step - accuracy: 0.2086 - loss: 3.8516
Epoch 12/50
2499/24

In [ ]:
# Save the model after training
model.save('/content/lstm.h5')

In [ ]:
import pickle

# Save the tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
%%writefile app.py
import streamlit as st
import numpy as np
import tensorflow as tf
import pickle

# Load the tokenizer
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)


# Load your trained LSTM model
model = tf.keras.models.load_model('/content/lstm.h5')

# Create a Streamlit app
st.title("Shakespeare Next Word Predictor")
st.write("Type a sentence and get the next word prediction:")

# Text input for the user to enter a sentence
input_text = st.text_input("Enter a partial sentence:")

# Function to predict the next word
def predict_next_word(text):
    sequences = tokenizer.texts_to_sequences([text])[0]
    if len(sequences) < 5:  # Adjust to your sequence length
        return "Input is too short."

    input_sequence = sequences[-5:]  # Use the last 3 words
    input_sequence = np.array(input_sequence).reshape(1, -1)

    predictions = model.predict(input_sequence, verbose=0)
    predicted_word_index = np.argmax(predictions)
    predicted_word = tokenizer.index_word.get(predicted_word_index, "")

    return predicted_word

if input_text:
    next_word = predict_next_word(input_text)
    st.write(f"Predicted next word: **{next_word}**")

Overwriting app.py


In [ ]:
!wget -q -O - ipv4.icanhazip.com

35.234.56.210


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.234.56.210:8501

your url is: https://better-laws-listen.loca.lt
2024-09-29 12:23:41.418951: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-29 12:23:41.438329: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-29 12:23:41.443439: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-29 12:23:42.483132: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
  Stopping...
^C
